In [158]:
import numpy as np 
import pandas as pd 


import os
from pmdarima.arima import auto_arima
import psycopg2 as pg

import pandas.io.sql as pd_sql
from sklearn.model_selection import train_test_split



In [159]:
import psycopg2 as pg

connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'names',    # DB that we are connecting to
    'port': 5432          # port for psql
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)
from sqlalchemy import create_engine
engine = create_engine('postgresql://hansremy:Hryr1321@@localhost:5432/Project5')

In [160]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [161]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

In [162]:
rootPathSourceData = './Data/smart-meters-in-london/'

In [163]:
weatherHourlyPath = rootPathSourceData + 'weather_hourly_darksky.csv'
df_weatherHourly = pd.read_csv(weatherHourlyPath, header=0, sep=',')

In [164]:
df_weatherHourly

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy
...,...,...,...,...,...,...,...,...,...,...,...,...
21160,12.68,247,7.39,2014-02-15 19:00:00,3.06,997.07,3.91,6.08,rain,partly-cloudy-night,0.74,Partly Cloudy
21161,13.78,249,6.56,2014-02-15 20:00:00,2.76,998.15,3.03,5.61,rain,clear-night,0.77,Clear
21162,14.31,248,6.47,2014-02-15 21:00:00,2.71,999.28,3.06,5.25,rain,clear-night,0.77,Clear
21163,14.31,248,5.96,2014-02-15 22:00:00,2.69,1000.33,2.68,4.69,rain,clear-night,0.80,Clear


In [165]:
# Postgres info to connect
'''
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'names',    # DB that we are connecting to
    'port': 5432          # port for psql
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)
from sqlalchemy import create_engine
engine = create_engine('postgresql://hansremy:Hryr1321@@localhost:5432/Project5')
df_weatherHourly.to_sql('WeatherHourly', engine)
'''

"\nconnection_args = {\n    'host': 'localhost',  # We are connecting to our _local_ version of psql\n    'dbname': 'names',    # DB that we are connecting to\n    'port': 5432          # port for psql\n}\n\n# We will talk about this magic Python trick!\nconnection = pg.connect(**connection_args)\nfrom sqlalchemy import create_engine\nengine = create_engine('postgresql://hansremy:Hryr1321@@localhost:5432/Project5')\ndf_weatherHourly.to_sql('WeatherHourly', engine)\n"

In [166]:
#df_Adversity = pd.read_pickle(('./Data/AdversityGroup.pkl') )
#df_Affluent = pd.read_pickle(('./Data/AffluentGroup.pkl') )
#df_Comfortable = pd.read_pickle(('./Data/ComfortableGroup.pkl') )

In [173]:
df_Adversity = pd.read_sql_table('Adversity',engine)
df_Affluent = pd.read_sql_table('Affluent',engine)
df_Comfortable = pd.read_sql_table('Comfortable',engine)
#df_Adversity.drop('index',axis=1,inplace=True)
#df_Affluent.drop('index',axis=1,inplace=True)
#df_Comfortable.drop('index',axis=1,inplace=True)

In [175]:
df_Affluent.dropna()
df_Adversity.dropna()
df_Comfortable.dropna()

,tstp,energy,temperature
609,2011-11-23 09:30:00.0000000,0.41400,3.42
610,2011-11-23 10:30:00.0000000,0.09950,4.24
611,2011-11-23 11:30:00.0000000,0.13340,5.94
612,2011-11-23 12:30:00.0000000,0.11020,7.25
613,2011-11-23 13:30:00.0000000,0.18500,8.29
...,...,...,...
40327,2014-02-27 20:00:00.0000000,0.33923,3.47
40328,2014-02-27 21:00:00.0000000,0.32581,2.50
40329,2014-02-27 22:00:00.0000000,0.29109,1.41
40330,2014-02-27 23:00:00.0000000,0.24259,1.42


In [168]:
df_Comfortable

,tstp,energy,temperature
0,2012-12-18 15:13:26.0000000,NaN,NaN
1,2012-12-18 15:13:27.0000000,NaN,NaN
2,2012-12-18 15:13:28.0000000,NaN,NaN
3,2012-12-18 15:13:29.0000000,NaN,NaN
4,2012-12-18 15:13:30.0000000,NaN,NaN
...,...,...,...
40327,2014-02-27 20:00:00.0000000,0.33923,3.47
40328,2014-02-27 21:00:00.0000000,0.32581,2.50
40329,2014-02-27 22:00:00.0000000,0.29109,1.41
40330,2014-02-27 23:00:00.0000000,0.24259,1.42


In [ ]:
df_ukHoliday = pd.read_sql_table('UKHoliday',engine)

In [ ]:
df_ukHoliday

In [ ]:
df_ukHoliday['Date'] = pd.to_datetime(df_ukHoliday['Date'])

In [ ]:
df_ukHoliday

In [ ]:
def makeHolidayDFProphet(df_Holiday):
    df = df_Holiday.groupby('Description')['Date'].apply(list)
    df = df_Holiday.groupby('Description')['Date'].apply(list).reset_index(name='ds')
    
  
    return df

In [ ]:
df_Holidays = makeHolidayDFProphet(df_ukHoliday)

In [ ]:
df_Holidays

In [ ]:
df_Holidays.columns = ['holiday', 'ds']

In [170]:
#df_Adversity = df_Adversity.set_index("tstp")
#df_Affluent = df_Affluent.set_index("tstp")
#df_Comfortable = df_Comfortable.set_index("tstp")
df_Adversity.index = df_Adversity.tstp.astype('datetime64[ns]')
df_Affluent.index = df_Affluent.tstp.astype('datetime64[ns]')
df_Comfortable.index = df_Comfortable.tstp.astype('datetime64[ns]')


In [171]:
df_Adversity = df_Adversity[df_Adversity["energy"] != "Null"]
df_Adversity["energy"] = df_Adversity["energy"].astype("float64")
df_Affluent = df_Affluent[df_Affluent["energy"] != "Null"]
df_Affluent["energy"] = df_Affluent["energy"].astype("float64")
df_Comfortable = df_Comfortable[df_Comfortable["energy"] != "Null"]
df_Comfortable["energy"] = df_Comfortable["energy"].astype("float64")


/Users/hansremy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [172]:
df_Comfortable

,tstp,energy,temperature
tstp,,,
2012-12-18 15:13:26,2012-12-18 15:13:26.0000000,NaN,NaN
2012-12-18 15:13:27,2012-12-18 15:13:27.0000000,NaN,NaN
2012-12-18 15:13:28,2012-12-18 15:13:28.0000000,NaN,NaN
2012-12-18 15:13:29,2012-12-18 15:13:29.0000000,NaN,NaN
2012-12-18 15:13:30,2012-12-18 15:13:30.0000000,NaN,NaN
...,...,...,...
2014-02-27 20:00:00,2014-02-27 20:00:00.0000000,0.33923,3.47
2014-02-27 21:00:00,2014-02-27 21:00:00.0000000,0.32581,2.50
2014-02-27 22:00:00,2014-02-27 22:00:00.0000000,0.29109,1.41


In [ ]:
df_Adversity.plot(y="energy", figsize=(12, 4))

In [ ]:
df_Affluent.plot(y="energy", figsize=(12, 4))

In [ ]:
df_Comfortable.plot(y="energy", figsize=(12, 4))

In [157]:
df_Comfortable.loc[:, df_Comfortable.isna().any()]

,y,temperature
tstp,,
2011-11-24 00:30:00,0.16757,7.37
2011-11-24 01:30:00,0.15543,7.51
2011-11-24 02:30:00,0.13329,7.01
2011-11-24 03:30:00,0.10229,7.32
2011-11-24 04:30:00,0.08714,7.01
...,...,...
2014-02-27 20:00:00,0.33923,3.47
2014-02-27 21:00:00,0.32581,2.50
2014-02-27 22:00:00,0.29109,1.41


In [153]:
print(df_Adversity)

                                            tstp   energy  temperature
tstp                                                                  
2011-11-23 09:30:00  2011-11-23 09:30:00.0000000  0.14700         3.42
2011-11-23 10:30:00  2011-11-23 10:30:00.0000000  0.06300         4.24
2011-11-23 11:30:00  2011-11-23 11:30:00.0000000  0.12000         5.94
2011-11-23 12:30:00  2011-11-23 12:30:00.0000000  0.08950         7.25
2011-11-23 13:30:00  2011-11-23 13:30:00.0000000  0.34500         8.29
...                                          ...      ...          ...
2014-02-27 20:00:00  2014-02-27 20:00:00.0000000  0.27816         3.47
2014-02-27 21:00:00  2014-02-27 21:00:00.0000000  0.26860         2.50
2014-02-27 22:00:00  2014-02-27 22:00:00.0000000  0.23544         1.41
2014-02-27 23:00:00  2014-02-27 23:00:00.0000000  0.20078         1.42
2014-02-28 00:00:00  2014-02-28 00:00:00.0000000  0.17575         1.47

[40359 rows x 3 columns]


In [154]:
def MakeProphetModel(df):
   
    train_size = int(0.8 * len(df))
    dfy = df['energy']
    print(df)
    df.columns = ['ds','y','temperature']
    print(df)
    X_train, X_test, y_train, y_test = train_test_split(df, dfy, test_size=0.8)
    
    model = Prophet(yearly_seasonality=True)
    
    model.add_seasonality(name="monthly", period=30.5, fourier_order=5)
    model.add_regressor('temperature')
    model.fit(X_train)
    
    return model,X_train, X_test

In [155]:
'''
def MakeProphetModelBack(df):
    train_size = int(0.8 * len(df))
    dfy = df['energy']
    dfX = df.drop('energy',1)
    #X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.8)
    X_train, X_test = df[:train_size].drop('energy',1), df[train_size:].drop('energy',1)
    
    y_train, y_test = df[:train_size]["energy"].values, df[train_size:]["energy"].values
    #train_df = pd.concat([pd.Series(X_train), pd.Series(y_train)], axis=1, keys=["ds", "y","temperature"])
    #test_df = pd.concat([pd.Series(X_test), pd.Series([0]*len(y_test))], axis=1, keys=["ds", "y","temperature"])
    train_df = pd.concat([pd.Series(X_train), pd.Series(y_train)], axis=1)
    test_df = pd.concat([pd.Series(X_test), pd.Series([0]*len(y_test))], axis=1)
    answer_df = pd.concat([pd.Series(X_test), pd.Series(y_test)], axis=1, keys=["ds", "y"])
    model = Prophet(yearly_seasonality=True)
    print(X_train)
    print(train_df)
    model.add_seasonality(name="monthly", period=30.5, fourier_order=5)
    model.add_regressor('temperature')
    model.fit(train_df)
    
    return model,train_df,test_df,answer_df
'''

'\ndef MakeProphetModelBack(df):\n    train_size = int(0.8 * len(df))\n    dfy = df[\'energy\']\n    dfX = df.drop(\'energy\',1)\n    #X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.8)\n    X_train, X_test = df[:train_size].drop(\'energy\',1), df[train_size:].drop(\'energy\',1)\n    \n    y_train, y_test = df[:train_size]["energy"].values, df[train_size:]["energy"].values\n    #train_df = pd.concat([pd.Series(X_train), pd.Series(y_train)], axis=1, keys=["ds", "y","temperature"])\n    #test_df = pd.concat([pd.Series(X_test), pd.Series([0]*len(y_test))], axis=1, keys=["ds", "y","temperature"])\n    train_df = pd.concat([pd.Series(X_train), pd.Series(y_train)], axis=1)\n    test_df = pd.concat([pd.Series(X_test), pd.Series([0]*len(y_test))], axis=1)\n    answer_df = pd.concat([pd.Series(X_test), pd.Series(y_test)], axis=1, keys=["ds", "y"])\n    model = Prophet(yearly_seasonality=True)\n    print(X_train)\n    print(train_df)\n    model.add_seasonality(name="mon

In [156]:
modelAdv, train_dfAdv,test_dfAdv = MakeProphetModel(df_Adversity)
modelComf, train_dfComf,test_dfComf = MakeProphetModel(df_Comfortable)
modelAffl, train_dfAffl,test_dfAffl= MakeProphetModel(df_Affluent)

                                            tstp   energy  temperature
tstp                                                                  
2011-11-23 09:30:00  2011-11-23 09:30:00.0000000  0.14700         3.42
2011-11-23 10:30:00  2011-11-23 10:30:00.0000000  0.06300         4.24
2011-11-23 11:30:00  2011-11-23 11:30:00.0000000  0.12000         5.94
2011-11-23 12:30:00  2011-11-23 12:30:00.0000000  0.08950         7.25
2011-11-23 13:30:00  2011-11-23 13:30:00.0000000  0.34500         8.29
...                                          ...      ...          ...
2014-02-27 20:00:00  2014-02-27 20:00:00.0000000  0.27816         3.47
2014-02-27 21:00:00  2014-02-27 21:00:00.0000000  0.26860         2.50
2014-02-27 22:00:00  2014-02-27 22:00:00.0000000  0.23544         1.41
2014-02-27 23:00:00  2014-02-27 23:00:00.0000000  0.20078         1.42
2014-02-28 00:00:00  2014-02-28 00:00:00.0000000  0.17575         1.47

[40359 rows x 3 columns]
                                              ds   

ValueError: Found NaN in column 'temperature'

In [ ]:
forecast = model.predict(test_df)

In [ ]:
forecast.head()


In [ ]:
model.plot(forecast)

model.plot(forecast).savefig('./Graphs/mvpforcast.png');

In [ ]:
model.plot_components(forecast)
model.plot_components(forecast).savefig('./Graphs/mvpcompforcast.png');

In [ ]:
df_cv = cross_validation(model, horizon="60 days")
df_cv.head()

In [ ]:
df_p = performance_metrics(df_cv)
df_p

In [ ]:
plt.plot(answer_df["ds"], answer_df["y"])
plt.plot(forecast["ds"], forecast["yhat"])

In [ ]:
# We have to add week and month seasonality
model = Prophet(weekly_seasonality=True)
model.add_seasonality(name="monthly", period=30.5, fourier_order=5)
model.fit(train_df)

In [ ]:
forecast = model.predict(test_df)
forecast.head()

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)
